# Dermatology Classification Problem

**Group number:** XX

* **Devora Cavaleiro** - 20230974
* **Jaime**
* **Maria**
* **Carlos**
* **Taha**

# 1. Import libraries

In [18]:
import requests
from PIL import Image
from PIL import UnidentifiedImageError
import os
from urllib.parse import urlparse
import shutil
import io

import numpy as np
import pandas as pd
import csv
import re

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 2. Dataset

The dataset of images was taken from the “FITZPATRICK17” repository, and is a combination of two dermatology datasets, 
“DermaAmin” and “ATLAS Dermatologico”. The goal is to label the specific disease, by building a deep learning model to address this classification problem.

The dataset has the following columns:

* **md5hash:** cryptographic hash algorithm used to generate a 128-bit digest from a string of any length.
* **fitzpatrick_scale:** classification system based on the amount of melanin present in the skin, how the skin reacts to sun exposure, and the skin's tendency to sunburn or tan.
* **fitzpatrick_centaur:** XXX
* **label:** disease's diagnoses.
* **nine_partition_label:** Categorize diseases in 9 groups (XXX).
* **three_partition_label:** Categorize diseases in 3 groups (XXX).
* **qc:** quality control - indicates if the diagnose is reliable or not.
* **url:** web address pointing to the location of the image on the internet.
* **url_alphanum:** process version of the URL.


*References:*
- https://www.simplilearn.com/tutorials/cyber-security-tutorial/md5-algorithm;
- https://www.usdermatologypartners.com/fitzpatrick-scale/;

In [19]:
df = pd.read_csv('./fitzpatrick17k.csv')
df

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
...,...,...,...,...,...,...,...,...,...
16572,cd90e491ddaa92f0f4eb07f73aa09f64,5,5,scleroderma,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4529,httpwwwatlasdermatologicocombrimgimageId4529.jpg
16573,5f1ed6de6a9110d7dc580a6a0312af63,4,5,pityriasis rosea,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=5630,httpwwwatlasdermatologicocombrimgimageId5630.jpg
16574,f198aaf1f0550c2464b285454d34926e,3,3,myiasis,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4609,httpwwwatlasdermatologicocombrimgimageId4609.jpg
16575,6214de2e915835014235a1839cbc5938,-1,5,vitiligo,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=7930,httpwwwatlasdermatologicocombrimgimageId7930.jpg


Our target variable is the *'label'* column, that has the name of the diseases.

In [20]:
df.label.unique()

array(['drug induced pigmentary changes', 'photodermatoses',
       'dermatofibroma', 'psoriasis', 'kaposi sarcoma',
       'neutrophilic dermatoses', 'granuloma annulare',
       'nematode infection', 'allergic contact dermatitis',
       'necrobiosis lipoidica', 'hidradenitis', 'melanoma',
       'acne vulgaris', 'sarcoidosis', 'xeroderma pigmentosum',
       'actinic keratosis', 'scleroderma', 'syringoma', 'folliculitis',
       'pityriasis lichenoides chronica', 'porphyria',
       'dyshidrotic eczema', 'seborrheic dermatitis', 'prurigo nodularis',
       'acne', 'neurofibromatosis', 'eczema', 'pediculosis lids',
       'basal cell carcinoma', 'pityriasis rubra pilaris',
       'pityriasis rosea', 'livedo reticularis',
       'stevens johnson syndrome', 'erythema multiforme',
       'acrodermatitis enteropathica', 'epidermolysis bullosa',
       'dermatomyositis', 'urticaria', 'basal cell carcinoma morpheiform',
       'vitiligo', 'erythema nodosum', 'lupus erythematosus',
       '

In [21]:
# df.label.unique() ----- count labels

This column *'qc'* has over 96% of missing values so we won't use it for training our model, because it doesn't have enought information.

In [22]:
nan_counts = df.isna().sum()
nan_counts

md5hash                      0
fitzpatrick_scale            0
fitzpatrick_centaur          0
label                        0
nine_partition_label         0
three_partition_label        0
qc                       16073
url                         41
url_alphanum                 0
dtype: int64

In [23]:
df.qc.unique()

array([nan, '1 Diagnostic', '5 Potentially', '3 Wrongly labelled',
       '2 Characteristic', '4 Other'], dtype=object)

In [24]:
df.drop(columns=['qc'], inplace=True)

When checking for **missing values**, there are some in the columns corresponding to *'qc'* and *'url'*.

In [25]:
nan_counts = df.isna().sum()
nan_counts

md5hash                   0
fitzpatrick_scale         0
fitzpatrick_centaur       0
label                     0
nine_partition_label      0
three_partition_label     0
url                      41
url_alphanum              0
dtype: int64

In [26]:
missing_url_rows = df[df['url'].isna()]
missing_url_rows

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,url,url_alphanum
145,fc63008d9d80b373d2d8f84dfd034446,3,4,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
349,480fd80167d7865ce9aafe14aba4ec9d,5,-1,melanoma,malignant melanoma,malignant,NaN,httpwwwdermaamincomsiteimagesclinicalpicooralm...
535,acbc68bd9e35591b8344ab8b5e51f458,3,3,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
868,93997108c3ae3beacc108f8829dc2302,4,1,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
879,9d994f1278a9788612fa2af179328c31,6,6,melanoma,malignant melanoma,malignant,NaN,httpwwwdermaamincomsiteimagesclinicalpicooralm...
918,6712381a925124e968bcac89481c90e0,2,2,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
1322,bb23fa10c71a57d17a8c7eb4773db237,3,3,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
1958,d653edb6aa702f095a9b00550738a821,1,-1,nevocytic nevus,benign melanocyte,benign,NaN,httpwwwdermaamincomsiteimageshistopicjjunction...
1972,8cfe54501fe00ddc74096b7cd67aa1e1,2,2,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...
2078,85e11dd1c6f60d02f16516fdfac9c7f7,2,1,lichen simplex,inflammatory,non-neoplastic,NaN,httpwwwdermaamincomsiteimagesclinicalpicLLiche...


In [27]:
# drop NaN from dataset
df.dropna(inplace=True)

In [28]:
nan_counts = df.isna().sum()
nan_counts

md5hash                  0
fitzpatrick_scale        0
fitzpatrick_centaur      0
label                    0
nine_partition_label     0
three_partition_label    0
url                      0
url_alphanum             0
dtype: int64

In [29]:
df

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
...,...,...,...,...,...,...,...,...
16572,cd90e491ddaa92f0f4eb07f73aa09f64,5,5,scleroderma,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=4529,httpwwwatlasdermatologicocombrimgimageId4529.jpg
16573,5f1ed6de6a9110d7dc580a6a0312af63,4,5,pityriasis rosea,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=5630,httpwwwatlasdermatologicocombrimgimageId5630.jpg
16574,f198aaf1f0550c2464b285454d34926e,3,3,myiasis,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=4609,httpwwwatlasdermatologicocombrimgimageId4609.jpg
16575,6214de2e915835014235a1839cbc5938,-1,5,vitiligo,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=7930,httpwwwatlasdermatologicocombrimgimageId7930.jpg


# 3. Dowload the images

Define a function (download_image) to download images from a given URL and save them to the specified filename.

In [15]:
def download_image(image_url, file_place):
    """
    Downloads images from URLs.

    Inputs:
        image_url(str): URL of the image to download.
        file_place(str): Path where the downloaded image will be saved.

    Outputs:
        None.
    """
    # Step 1: Send a GET request to the provided image URL
    res = requests.get(image_url, stream=True, headers={'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"})
    
    # Step 2: Check if the request was successful (HTTP status code 200)
    if res.status_code == 200:
        # Step 3: If successful, open the file at the specified location in binary write mode
        with open(file_place, 'wb') as f:
          shutil.copyfileobj(res.raw, f) # Step 4: Copy the content of the response to the file
        # Step 5: Print the status of the download
        print('Image successfully downloaded', file_place)
    else:
        print("Image coulnd't be retrieved")

Define a function (download_image) to download images from a given URL and save them to the specified filename.

In [16]:
def download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name):
    """
    Downloads images from URLs listed in a CSV file.
    
    Inputs:
        csv_file (str): Path to the CSV file containing image URLs and labels.
        url_column_name (str): Name of the column containing image URLs.
        output_folder (str): Path to the folder where images will be saved.
        label_column_name (str): Name of the column containing labels.

    Outputs:
        None.
    """
    data = pd.read_csv(csv_file)  # Step1: Read CSV
    for index, row in data.iterrows(): # Step 2: Iterate throught a loop for extract the url from the column
        image_url = row[url_column_name]
        label = row[label_column_name]
        if isinstance(image_url, str) and not pd.isnull(image_url):  # Step 3: Check for NaN in URL column
            filename = f"{output_folder}/{label}_{index}.jpg" # Step 4: generate a file for the downloaded image 
            download_image(image_url, filename) # Step 5: The download_image function downloads the image from the URL and saves it to the specified file
        else:
            print(f"Skipping row {index} due to missing or invalid URL.")

In [30]:
# with skip index
def download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index):
    """
    Downloads images from URLs listed in a CSV file.
    
    Inputs:
        csv_file (str): Path to the CSV file containing image URLs and labels.
        url_column_name (str): Name of the column containing image URLs.
        output_folder (str): Path to the folder where images will be saved.
        label_column_name (str): Name of the column containing labels.
        skip_index (int): Index to skip downloading until.

    Outputs:
        None.
    """
    data = pd.read_csv(csv_file)  # Step1: Read CSV
    for index, row in data.iterrows(): # Step 2: Iterate throught a loop for extract the url from the column
        if index <= skip_index:
            continue
        image_url = row[url_column_name]
        label = row[label_column_name]
        if isinstance(image_url, str) and not pd.isnull(image_url):  # Step 3: Check for NaN in URL column
            filename = f"{output_folder}/{label}_{index}.jpg" # Step 4: generate a file for the downloaded image 
            download_image(image_url, filename) # Step 5: The download_image function downloads the image from the URL and saves it to the specified file
        else:
            print(f"Skipping row {index} due to missing or invalid URL.")

Use the functions created to download the images:

In [17]:
csv_file = './fitzpatrick17k.csv'  # CSV file path
url_column_name = 'url'
label_column_name = 'label'
output_folder = 'project_pic'  # folder where to save downloaded images
skip_index = 9759
download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index)

Image successfully downloaded project_pic/drug induced pigmentary changes_0.jpg
Image successfully downloaded project_pic/photodermatoses_1.jpg
Image successfully downloaded project_pic/dermatofibroma_2.jpg
Image successfully downloaded project_pic/psoriasis_3.jpg
Image successfully downloaded project_pic/psoriasis_4.jpg
Image successfully downloaded project_pic/kaposi sarcoma_5.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_6.jpg
Image successfully downloaded project_pic/granuloma annulare_7.jpg
Image successfully downloaded project_pic/nematode infection_8.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_9.jpg
Image successfully downloaded project_pic/necrobiosis lipoidica_10.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_11.jpg
Image successfully downloaded project_pic/hidradenitis_12.jpg
Image successfully downloaded project_pic/melanoma_13.jpg
Image successfully downloaded project_pic/acne vulgaris_14.jpg
Image

Image successfully downloaded project_pic/mucinosis_124.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_125.jpg
Image successfully downloaded project_pic/eczema_126.jpg
Image successfully downloaded project_pic/keratosis pilaris_127.jpg
Image successfully downloaded project_pic/urticaria_128.jpg
Image successfully downloaded project_pic/keloid_129.jpg
Image successfully downloaded project_pic/psoriasis_130.jpg
Image successfully downloaded project_pic/tuberous sclerosis_131.jpg
Image successfully downloaded project_pic/sarcoidosis_132.jpg
Image successfully downloaded project_pic/lichen planus_133.jpg
Image successfully downloaded project_pic/milia_134.jpg
Image successfully downloaded project_pic/striae_135.jpg
Image successfully downloaded project_pic/acquired autoimmune bullous diseaseherpes gestationis_136.jpg
Image successfully downloaded project_pic/basal cell carcinoma_137.jpg
Image successfully downloaded project_pic/fixed eruptions_138.jpg
Image succe

Image successfully downloaded project_pic/acne_247.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_248.jpg
Image successfully downloaded project_pic/pyogenic granuloma_249.jpg
Image successfully downloaded project_pic/langerhans cell histiocytosis_250.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_251.jpg
Image successfully downloaded project_pic/psoriasis_252.jpg
Image successfully downloaded project_pic/lyme disease_253.jpg
Image successfully downloaded project_pic/lymphangioma_254.jpg
Image successfully downloaded project_pic/port wine stain_255.jpg
Image successfully downloaded project_pic/malignant melanoma_256.jpg
Image successfully downloaded project_pic/folliculitis_257.jpg
Image successfully downloaded project_pic/acne vulgaris_258.jpg
Image successfully downloaded project_pic/psoriasis_259.jpg
Image successfully downloaded project_pic/porphyria_260.jpg
Image successfully downloaded project_pic/actinic keratosis_261.jpg
Image su

Image successfully downloaded project_pic/keloid_370.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_371.jpg
Image successfully downloaded project_pic/photodermatoses_372.jpg
Image successfully downloaded project_pic/actinic keratosis_373.jpg
Image successfully downloaded project_pic/actinic keratosis_374.jpg
Image successfully downloaded project_pic/scleroderma_375.jpg
Image successfully downloaded project_pic/incontinentia pigmenti_376.jpg
Image successfully downloaded project_pic/scleroderma_377.jpg
Image successfully downloaded project_pic/sarcoidosis_378.jpg
Image successfully downloaded project_pic/malignant melanoma_379.jpg
Image successfully downloaded project_pic/solid cystic basal cell carcinoma_380.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_381.jpg
Image successfully downloaded project_pic/rosacea_382.jpg
Image successfully downloaded project_pic/drug induced pigmentary changes_383.jpg
Image successfully downloaded project

Image successfully downloaded project_pic/basal cell carcinoma_493.jpg
Image successfully downloaded project_pic/malignant melanoma_494.jpg
Image successfully downloaded project_pic/dariers disease_495.jpg
Image successfully downloaded project_pic/dariers disease_496.jpg
Image successfully downloaded project_pic/lichen planus_497.jpg
Image successfully downloaded project_pic/granuloma annulare_498.jpg
Image successfully downloaded project_pic/myiasis_499.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_500.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_501.jpg
Image successfully downloaded project_pic/granuloma annulare_502.jpg
Image successfully downloaded project_pic/seborrheic dermatitis_503.jpg
Image successfully downloaded project_pic/urticaria pigmentosa_504.jpg
Image successfully downloaded project_pic/lupus erythematosus_505.jpg
Image successfully downloaded project_pic/dermatofibroma_506.jpg
Image successfully downlo

...

Image successfully downloaded project_pic/urticaria_9305.jpg
Image successfully downloaded project_pic/becker nevus_9306.jpg
Image successfully downloaded project_pic/scleroderma_9307.jpg
Image successfully downloaded project_pic/porokeratosis of mibelli_9308.jpg
Image successfully downloaded project_pic/cheilitis_9309.jpg
Image successfully downloaded project_pic/solid cystic basal cell carcinoma_9310.jpg
Image successfully downloaded project_pic/scabies_9311.jpg
Image successfully downloaded project_pic/vitiligo_9312.jpg
Image successfully downloaded project_pic/cheilitis_9313.jpg
Image successfully downloaded project_pic/malignant melanoma_9314.jpg
Image successfully downloaded project_pic/striae_9315.jpg
Image successfully downloaded project_pic/pediculosis lids_9316.jpg
Image successfully downloaded project_pic/lichen planus_9317.jpg
Image successfully downloaded project_pic/pediculosis lids_9318.jpg
Image successfully downloaded project_pic/fordyce spots_9319.jpg
Image successful

Image successfully downloaded project_pic/acne_9424.jpg
Image successfully downloaded project_pic/epidermal nevus_9425.jpg
Image successfully downloaded project_pic/mucous cyst_9426.jpg
Image successfully downloaded project_pic/seborrheic keratosis_9427.jpg
Image successfully downloaded project_pic/acne_9428.jpg
Image successfully downloaded project_pic/melanoma_9429.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_9430.jpg
Image successfully downloaded project_pic/tungiasis_9431.jpg
Image successfully downloaded project_pic/pyogenic granuloma_9432.jpg
Image successfully downloaded project_pic/epidermal nevus_9433.jpg
Image successfully downloaded project_pic/mucous cyst_9434.jpg
Image successfully downloaded project_pic/folliculitis_9435.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_9436.jpg
Image successfully downloaded project_pic/striae_9437.jpg
Image successfully downloaded project_pic/scabies_9438.jpg
Image successfully downloaded 

Image successfully downloaded project_pic/livedo reticularis_9544.jpg
Image successfully downloaded project_pic/scleromyxedema_9545.jpg
Image successfully downloaded project_pic/necrobiosis lipoidica_9546.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_9547.jpg
Image successfully downloaded project_pic/pediculosis lids_9548.jpg
Image successfully downloaded project_pic/acanthosis nigricans_9549.jpg
Image successfully downloaded project_pic/psoriasis_9550.jpg
Image successfully downloaded project_pic/drug induced pigmentary changes_9551.jpg
Image successfully downloaded project_pic/psoriasis_9552.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_9553.jpg
Image successfully downloaded project_pic/acne vulgaris_9554.jpg
Image successfully downloaded project_pic/lyme disease_9555.jpg
Image successfully downloaded project_pic/acne_9556.jpg
Image successfully downloaded project_pic/rhinophyma_9557.jpg
Image successfully downloaded project_pic/drug

Image successfully downloaded project_pic/neurodermatitis_9664.jpg
Image successfully downloaded project_pic/folliculitis_9665.jpg
Image successfully downloaded project_pic/dermatofibroma_9666.jpg
Image successfully downloaded project_pic/urticaria pigmentosa_9667.jpg
Image successfully downloaded project_pic/acne_9668.jpg
Image successfully downloaded project_pic/paronychia_9669.jpg
Image successfully downloaded project_pic/syringoma_9670.jpg
Image successfully downloaded project_pic/photodermatoses_9671.jpg
Image successfully downloaded project_pic/pyogenic granuloma_9672.jpg
Image successfully downloaded project_pic/acne vulgaris_9673.jpg
Image successfully downloaded project_pic/vitiligo_9674.jpg
Image successfully downloaded project_pic/superficial spreading melanoma ssm_9675.jpg
Image successfully downloaded project_pic/acne vulgaris_9676.jpg
Image successfully downloaded project_pic/incontinentia pigmenti_9677.jpg
Image successfully downloaded project_pic/xeroderma pigmentosum_9

ConnectionError: HTTPSConnectionPool(host='www.dermaamin.com', port=443): Max retries exceeded with url: /site/images/clinical-pic/b/brown_recluse_spider_bite/brown_recluse_spider_bite3.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002356508B110>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [31]:
csv_file = './fitzpatrick17k.csv'  # CSV file path
url_column_name = 'url'
label_column_name = 'label'
output_folder = 'project_pic'  # folder where to save downloaded images
skip_index = 9759
download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index)

Image successfully downloaded project_pic/tick bite_9760.jpg
Image successfully downloaded project_pic/psoriasis_9761.jpg
Image successfully downloaded project_pic/lichen planus_9762.jpg
Image successfully downloaded project_pic/psoriasis_9763.jpg
Image successfully downloaded project_pic/urticaria pigmentosa_9764.jpg
Image successfully downloaded project_pic/neurofibromatosis_9765.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_9766.jpg
Image successfully downloaded project_pic/urticaria pigmentosa_9767.jpg
Image successfully downloaded project_pic/nematode infection_9768.jpg
Image successfully downloaded project_pic/folliculitis_9769.jpg
Image successfully downloaded project_pic/folliculitis_9770.jpg
Image successfully downloaded project_pic/mucinosis_9771.jpg
Image successfully downloaded project_pic/lyme disease_9772.jpg
Image successfully downloaded project_pic/pityriasis lichenoides chronica_9773.jpg
Image successfully downloaded project_pic/factitial dermat

Image successfully downloaded project_pic/mucinosis_9882.jpg
Image successfully downloaded project_pic/sarcoidosis_9883.jpg
Image successfully downloaded project_pic/neurofibromatosis_9884.jpg
Image successfully downloaded project_pic/basal cell carcinoma_9885.jpg
Image successfully downloaded project_pic/basal cell carcinoma morpheiform_9886.jpg
Image successfully downloaded project_pic/nevus sebaceous of jadassohn_9887.jpg
Image successfully downloaded project_pic/mycosis fungoides_9888.jpg
Image successfully downloaded project_pic/dyshidrotic eczema_9889.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_9890.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_9891.jpg
Image successfully downloaded project_pic/langerhans cell histiocytosis_9892.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_9893.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_9894.jpg
Image successfully downloa

Image successfully downloaded project_pic/porphyria_10002.jpg
Image successfully downloaded project_pic/dermatofibroma_10003.jpg
Image successfully downloaded project_pic/hidradenitis_10004.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_10005.jpg
Image successfully downloaded project_pic/nevocytic nevus_10006.jpg
Image successfully downloaded project_pic/folliculitis_10007.jpg
Image successfully downloaded project_pic/lichen planus_10008.jpg
Image successfully downloaded project_pic/neurodermatitis_10009.jpg
Image successfully downloaded project_pic/acne vulgaris_10010.jpg
Image successfully downloaded project_pic/milia_10011.jpg
Image successfully downloaded project_pic/dermatomyositis_10012.jpg
Image successfully downloaded project_pic/kaposi sarcoma_10013.jpg
Image successfully downloaded project_pic/kaposi sarcoma_10014.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_10015.jpg
Image successfully downloaded project_pic/telangiectases_1001

Image successfully downloaded project_pic/neutrophilic dermatoses_10122.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_10123.jpg
Image successfully downloaded project_pic/drug eruption_10124.jpg
Image successfully downloaded project_pic/juvenile xanthogranuloma_10125.jpg
Image successfully downloaded project_pic/lupus subacute_10126.jpg
Image successfully downloaded project_pic/erythema elevatum diutinum_10127.jpg
Image successfully downloaded project_pic/fixed eruptions_10128.jpg
Image successfully downloaded project_pic/psoriasis_10129.jpg
Image successfully downloaded project_pic/acanthosis nigricans_10130.jpg
Image successfully downloaded project_pic/porokeratosis of mibelli_10131.jpg
Image successfully downloaded project_pic/photodermatoses_10132.jpg
Image successfully downloaded project_pic/pilomatricoma_10133.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_10134.jpg
Image successfully downloaded project_pic/pyogenic granuloma_101

Image successfully downloaded project_pic/lupus subacute_10240.jpg
Image successfully downloaded project_pic/urticaria_10241.jpg
Image successfully downloaded project_pic/eczema_10242.jpg
Image successfully downloaded project_pic/urticaria pigmentosa_10243.jpg
Image successfully downloaded project_pic/psoriasis_10244.jpg
Image successfully downloaded project_pic/psoriasis_10245.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_10246.jpg
Image successfully downloaded project_pic/erythema elevatum diutinum_10247.jpg
Image successfully downloaded project_pic/lupus erythematosus_10248.jpg
Image successfully downloaded project_pic/sarcoidosis_10249.jpg
Image successfully downloaded project_pic/stasis edema_10250.jpg
Image successfully downloaded project_pic/erythema multiforme_10251.jpg
Image successfully downloaded project_pic/psoriasis_10252.jpg
Image successfully downloaded project_pic/porphyria_10253.jpg
Image successfully downloaded project_pic/paronychia_10254.jpg


...

Image successfully downloaded project_pic/granuloma annulare_12857.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_12858.jpg
Image successfully downloaded project_pic/dermatofibroma_12859.jpg
Image successfully downloaded project_pic/dariers disease_12860.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_12861.jpg
Image successfully downloaded project_pic/pediculosis lids_12862.jpg
Image successfully downloaded project_pic/photodermatoses_12863.jpg
Image successfully downloaded project_pic/scleromyxedema_12864.jpg
Image successfully downloaded project_pic/lichen amyloidosis_12865.jpg
Image successfully downloaded project_pic/lichen planus_12866.jpg
Image successfully downloaded project_pic/pityriasis rosea_12867.jpg
Image successfully downloaded project_pic/tungiasis_12868.jpg
Image successfully downloaded project_pic/prurigo nodularis_12869.jpg
Image successfully downloaded project_pic/pityriasis rosea_12870.jpg
Image successfully downloaded p

Image successfully downloaded project_pic/basal cell carcinoma_12976.jpg
Image successfully downloaded project_pic/mycosis fungoides_12977.jpg
Image successfully downloaded project_pic/erythema annulare centrifigum_12978.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_12979.jpg
Image successfully downloaded project_pic/psoriasis_12980.jpg
Image successfully downloaded project_pic/erythema multiforme_12981.jpg
Image successfully downloaded project_pic/acne_12982.jpg
Image successfully downloaded project_pic/porokeratosis actinic_12983.jpg
Image successfully downloaded project_pic/erythema annulare centrifigum_12984.jpg
Image successfully downloaded project_pic/nematode infection_12985.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_12986.jpg
Image successfully downloaded project_pic/lichen planus_12987.jpg
Image successfully downloaded project_pic/tungiasis_12988.jpg
Image successfully downloaded project_pic/lichen planus_12989.jpg
Image succ

Image successfully downloaded project_pic/lichen planus_13093.jpg
Image successfully downloaded project_pic/lichen planus_13094.jpg
Image successfully downloaded project_pic/dariers disease_13095.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_13096.jpg
Image successfully downloaded project_pic/cheilitis_13097.jpg
Image successfully downloaded project_pic/lichen planus_13098.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_13099.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_13100.jpg
Image successfully downloaded project_pic/scleroderma_13101.jpg
Image successfully downloaded project_pic/lichen amyloidosis_13102.jpg
Image successfully downloaded project_pic/lichen planus_13103.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13104.jpg
Image successfully downloaded project_pic/dariers disease_13105.jpg
Image successfully downloaded project_pic/mucinosis_13106.jpg
Image successfully

Image successfully downloaded project_pic/lichen planus_13209.jpg
Image successfully downloaded project_pic/mucous cyst_13210.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_13211.jpg
Image successfully downloaded project_pic/porokeratosis of mibelli_13212.jpg
Image successfully downloaded project_pic/ichthyosis vulgaris_13213.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13214.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_13215.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13216.jpg
Image successfully downloaded project_pic/porphyria_13217.jpg
Image successfully downloaded project_pic/folliculitis_13218.jpg
Image successfully downloaded project_pic/tungiasis_13219.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13220.jpg
Image successfully downloaded project_pic/lichen planus_13221.jpg
Image successfully downloaded project_pic/urticaria_13222.jpg
Image 

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Uma ligação existente foi forçada a fechar pelo anfitrião remoto', None, 10054, None))

In [32]:
csv_file = './fitzpatrick17k.csv'  # CSV file path
url_column_name = 'url'
label_column_name = 'label'
output_folder = 'project_pic'  # folder where to save downloaded images
skip_index = 13223
download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index)

Image successfully downloaded project_pic/psoriasis_13224.jpg
Image successfully downloaded project_pic/tungiasis_13225.jpg
Image successfully downloaded project_pic/lupus erythematosus_13226.jpg
Image successfully downloaded project_pic/photodermatoses_13227.jpg
Image successfully downloaded project_pic/folliculitis_13228.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13229.jpg
Image successfully downloaded project_pic/psoriasis_13230.jpg
Image successfully downloaded project_pic/lichen simplex_13231.jpg
Image successfully downloaded project_pic/scabies_13232.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_13233.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13234.jpg
Image successfully downloaded project_pic/lupus erythematosus_13235.jpg
Image successfully downloaded project_pic/erythema elevatum diutinum_13236.jpg
Image successfully downloaded project_pic/necrobiosis lipoidica_13237.jpg
Image successfully downloaded proj

Image successfully downloaded project_pic/pityriasis rosea_13341.jpg
Image successfully downloaded project_pic/pityriasis rosea_13342.jpg
Image successfully downloaded project_pic/porokeratosis actinic_13343.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13344.jpg
Image successfully downloaded project_pic/granuloma annulare_13345.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13346.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_13347.jpg
Image successfully downloaded project_pic/dariers disease_13348.jpg
Image successfully downloaded project_pic/pityriasis rosea_13349.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13350.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13351.jpg
Image successfully downloaded project_pic/scabies_13352.jpg
Image coulnd't be retrieved
Image successfully downloaded project_pic/behcets disease_13354.jpg
Image successfully downloaded pro

Image successfully downloaded project_pic/rhinophyma_13459.jpg
Image successfully downloaded project_pic/lichen planus_13460.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_13461.jpg
Image successfully downloaded project_pic/erythema multiforme_13462.jpg
Image successfully downloaded project_pic/pilar cyst_13463.jpg
Image successfully downloaded project_pic/lichen planus_13464.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13465.jpg
Image successfully downloaded project_pic/porokeratosis actinic_13466.jpg
Image successfully downloaded project_pic/neurotic excoriations_13467.jpg
Image successfully downloaded project_pic/fordyce spots_13468.jpg
Image successfully downloaded project_pic/scleromyxedema_13469.jpg
Image successfully downloaded project_pic/rhinophyma_13470.jpg
Image successfully downloaded project_pic/juvenile xanthogranuloma_13471.jpg
Image successfully downloaded project_pic/erythema elevatum diutinum_13472.jpg
Image successf

Image successfully downloaded project_pic/lichen planus_13576.jpg
Image successfully downloaded project_pic/fixed eruptions_13577.jpg
Image successfully downloaded project_pic/lichen simplex_13578.jpg
Image successfully downloaded project_pic/granuloma annulare_13579.jpg
Image successfully downloaded project_pic/porokeratosis of mibelli_13580.jpg
Image successfully downloaded project_pic/porokeratosis actinic_13581.jpg
Image successfully downloaded project_pic/seborrheic dermatitis_13582.jpg
Image successfully downloaded project_pic/erythema elevatum diutinum_13583.jpg
Image successfully downloaded project_pic/incontinentia pigmenti_13584.jpg
Image successfully downloaded project_pic/aplasia cutis_13585.jpg
Image successfully downloaded project_pic/mycosis fungoides_13586.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_13587.jpg
Image successfully downloaded project_pic/lupus erythematosus_13588.jpg
Image successfully downloaded project_pic/scabies_13589.jpg
Image 

Image successfully downloaded project_pic/pityriasis rubra pilaris_13694.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13695.jpg
Image successfully downloaded project_pic/porokeratosis actinic_13696.jpg
Image successfully downloaded project_pic/scabies_13697.jpg
Image successfully downloaded project_pic/erythema multiforme_13698.jpg
Image successfully downloaded project_pic/pediculosis lids_13699.jpg
Image successfully downloaded project_pic/lupus erythematosus_13700.jpg
Image successfully downloaded project_pic/seborrheic dermatitis_13701.jpg
Image successfully downloaded project_pic/melanoma_13702.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_13703.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13704.jpg
Image successfully downloaded project_pic/scleromyxedema_13705.jpg
Image successfully downloaded project_pic/basal cell carcinoma_13706.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_13707.jpg


...

Image successfully downloaded project_pic/hailey hailey disease_14519.jpg
Image successfully downloaded project_pic/mucous cyst_14520.jpg
Image successfully downloaded project_pic/psoriasis_14521.jpg
Image successfully downloaded project_pic/prurigo nodularis_14522.jpg
Image successfully downloaded project_pic/porokeratosis actinic_14523.jpg
Image successfully downloaded project_pic/vitiligo_14524.jpg
Image successfully downloaded project_pic/pediculosis lids_14525.jpg
Image successfully downloaded project_pic/scleroderma_14526.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_14527.jpg
Image successfully downloaded project_pic/fixed eruptions_14528.jpg
Image successfully downloaded project_pic/pityriasis rosea_14529.jpg
Image successfully downloaded project_pic/dermatofibroma_14530.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_14531.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_14532.jpg
Image successf

Image successfully downloaded project_pic/pilar cyst_14637.jpg
Image successfully downloaded project_pic/mycosis fungoides_14638.jpg
Image successfully downloaded project_pic/dariers disease_14639.jpg
Image successfully downloaded project_pic/mycosis fungoides_14640.jpg
Image successfully downloaded project_pic/necrobiosis lipoidica_14641.jpg
Image successfully downloaded project_pic/folliculitis_14642.jpg
Image successfully downloaded project_pic/basal cell carcinoma_14643.jpg
Image successfully downloaded project_pic/nematode infection_14644.jpg
Image successfully downloaded project_pic/scleroderma_14645.jpg
Image successfully downloaded project_pic/pediculosis lids_14646.jpg
Image successfully downloaded project_pic/neurofibromatosis_14647.jpg
Image successfully downloaded project_pic/basal cell carcinoma_14648.jpg
Image successfully downloaded project_pic/basal cell carcinoma_14649.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_14650.jpg
Image successfull

Image successfully downloaded project_pic/syringoma_14756.jpg
Image successfully downloaded project_pic/erythema annulare centrifigum_14757.jpg
Image successfully downloaded project_pic/pediculosis lids_14758.jpg
Image successfully downloaded project_pic/tungiasis_14759.jpg
Image successfully downloaded project_pic/stasis edema_14760.jpg
Image successfully downloaded project_pic/melanoma_14761.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_14762.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_14763.jpg
Image successfully downloaded project_pic/neurotic excoriations_14764.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_14765.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_14766.jpg
Image successfully downloaded project_pic/tungiasis_14767.jpg
Image successfully downloaded project_pic/xeroderma pigmentosum_14768.jpg
Image successfully downloaded project_pic/pityriasis rosea_14769.jpg
Image successfu

Image successfully downloaded project_pic/nematode infection_14872.jpg
Image successfully downloaded project_pic/neurofibromatosis_14873.jpg
Image successfully downloaded project_pic/scleroderma_14874.jpg
Image successfully downloaded project_pic/myiasis_14875.jpg
Image successfully downloaded project_pic/lichen planus_14876.jpg
Image successfully downloaded project_pic/pityriasis lichenoides chronica_14877.jpg
Image successfully downloaded project_pic/necrobiosis lipoidica_14878.jpg
Image successfully downloaded project_pic/photodermatoses_14879.jpg
Image successfully downloaded project_pic/basal cell carcinoma_14880.jpg
Image successfully downloaded project_pic/mycosis fungoides_14881.jpg
Image successfully downloaded project_pic/dariers disease_14882.jpg
Image successfully downloaded project_pic/lyme disease_14883.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_14884.jpg
Image successfully downloaded project_pic/pityriasis rosea_14885.jpg
Image successfully do

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [33]:
csv_file = './fitzpatrick17k.csv'  # CSV file path
url_column_name = 'url'
label_column_name = 'label'
output_folder = 'project_pic'  # folder where to save downloaded images
skip_index = 14888
download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index)

Image successfully downloaded project_pic/basal cell carcinoma_14889.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_14890.jpg
Image successfully downloaded project_pic/acne_14891.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_14892.jpg
Image successfully downloaded project_pic/lupus erythematosus_14893.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_14894.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_14895.jpg
Image successfully downloaded project_pic/juvenile xanthogranuloma_14896.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_14897.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_14898.jpg
Image successfully downloaded project_pic/vitiligo_14899.jpg
Image successfully downloaded project_pic/lupus erythematosus_14900.jpg
Image successfully downloaded project_pic/dariers disease_14901.jpg
Image successfully downl

Image successfully downloaded project_pic/mycosis fungoides_15008.jpg
Image successfully downloaded project_pic/vitiligo_15009.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_15010.jpg
Image successfully downloaded project_pic/lichen planus_15011.jpg
Image successfully downloaded project_pic/lupus erythematosus_15012.jpg
Image successfully downloaded project_pic/dariers disease_15013.jpg
Image successfully downloaded project_pic/scabies_15014.jpg
Image successfully downloaded project_pic/juvenile xanthogranuloma_15015.jpg
Image successfully downloaded project_pic/dariers disease_15016.jpg
Image successfully downloaded project_pic/melanoma_15017.jpg
Image successfully downloaded project_pic/tungiasis_15018.jpg
Image successfully downloaded project_pic/scabies_15019.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_15020.jpg
Image successfully downloaded project_pic/actinic keratosis_15021.jpg
Image successfully downloaded project_pic/erythema ann

Image successfully downloaded project_pic/squamous cell carcinoma_15125.jpg
Image successfully downloaded project_pic/mucinosis_15126.jpg
Image successfully downloaded project_pic/lupus erythematosus_15127.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_15128.jpg
Image successfully downloaded project_pic/basal cell carcinoma_15129.jpg
Image successfully downloaded project_pic/lichen planus_15130.jpg
Image successfully downloaded project_pic/lichen amyloidosis_15131.jpg
Image successfully downloaded project_pic/prurigo nodularis_15132.jpg
Image successfully downloaded project_pic/acanthosis nigricans_15133.jpg
Image successfully downloaded project_pic/psoriasis_15134.jpg
Image successfully downloaded project_pic/behcets disease_15135.jpg
Image successfully downloaded project_pic/dariers disease_15136.jpg
Image successfully downloaded project_pic/melanoma_15137.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_15138.jpg
Image successfully downlo

Image successfully downloaded project_pic/hailey hailey disease_15243.jpg
Image successfully downloaded project_pic/scabies_15244.jpg
Image successfully downloaded project_pic/ehlers danlos syndrome_15245.jpg
Image successfully downloaded project_pic/neurofibromatosis_15246.jpg
Image successfully downloaded project_pic/basal cell carcinoma_15247.jpg
Image successfully downloaded project_pic/melanoma_15248.jpg
Image successfully downloaded project_pic/hailey hailey disease_15249.jpg
Image successfully downloaded project_pic/urticaria_15250.jpg
Image successfully downloaded project_pic/psoriasis_15251.jpg
Image successfully downloaded project_pic/fixed eruptions_15252.jpg
Image successfully downloaded project_pic/pediculosis lids_15253.jpg
Image successfully downloaded project_pic/lupus erythematosus_15254.jpg
Image successfully downloaded project_pic/porphyria_15255.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_15256.jpg
Image successfully downloaded project_pic

Image successfully downloaded project_pic/lichen simplex_15361.jpg
Image successfully downloaded project_pic/kaposi sarcoma_15362.jpg
Image successfully downloaded project_pic/neurofibromatosis_15363.jpg
Image successfully downloaded project_pic/nematode infection_15364.jpg
Image successfully downloaded project_pic/porokeratosis actinic_15365.jpg
Image successfully downloaded project_pic/folliculitis_15366.jpg
Image successfully downloaded project_pic/dermatomyositis_15367.jpg
Image successfully downloaded project_pic/dariers disease_15368.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_15369.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_15370.jpg
Image successfully downloaded project_pic/nematode infection_15371.jpg
Image successfully downloaded project_pic/porokeratosis actinic_15372.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_15373.jpg
Image successfully downloaded project_pic/nematode infection_15374.jpg
Image

...

Image successfully downloaded project_pic/kaposi sarcoma_16184.jpg
Image successfully downloaded project_pic/porokeratosis actinic_16185.jpg
Image successfully downloaded project_pic/factitial dermatitis_16186.jpg
Image successfully downloaded project_pic/myiasis_16187.jpg
Image successfully downloaded project_pic/keloid_16188.jpg
Image successfully downloaded project_pic/scleromyxedema_16189.jpg
Image successfully downloaded project_pic/syringoma_16190.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_16191.jpg
Image successfully downloaded project_pic/melanoma_16192.jpg
Image successfully downloaded project_pic/fixed eruptions_16193.jpg
Image successfully downloaded project_pic/porokeratosis actinic_16194.jpg
Image successfully downloaded project_pic/lichen planus_16195.jpg
Image successfully downloaded project_pic/allergic contact dermatitis_16196.jpg
Image successfully downloaded project_pic/hailey hailey disease_16197.jpg
Image successfully downloaded project_p

Image successfully downloaded project_pic/lymphangioma_16302.jpg
Image successfully downloaded project_pic/nematode infection_16303.jpg
Image successfully downloaded project_pic/pityriasis rosea_16304.jpg
Image successfully downloaded project_pic/porokeratosis actinic_16305.jpg
Image successfully downloaded project_pic/neutrophilic dermatoses_16306.jpg
Image successfully downloaded project_pic/nematode infection_16307.jpg
Image successfully downloaded project_pic/tungiasis_16308.jpg
Image successfully downloaded project_pic/squamous cell carcinoma_16309.jpg
Image successfully downloaded project_pic/pilar cyst_16310.jpg
Image successfully downloaded project_pic/dyshidrotic eczema_16311.jpg
Image successfully downloaded project_pic/nematode infection_16312.jpg
Image successfully downloaded project_pic/folliculitis_16313.jpg
Image successfully downloaded project_pic/basal cell carcinoma_16314.jpg
Image successfully downloaded project_pic/myiasis_16315.jpg
Image successfully downloaded pro

Image successfully downloaded project_pic/scleromyxedema_16419.jpg
Image successfully downloaded project_pic/tuberous sclerosis_16420.jpg
Image successfully downloaded project_pic/tungiasis_16421.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_16422.jpg
Image successfully downloaded project_pic/calcinosis cutis_16423.jpg
Image successfully downloaded project_pic/pityriasis rubra pilaris_16424.jpg
Image successfully downloaded project_pic/basal cell carcinoma_16425.jpg
Image successfully downloaded project_pic/lichen planus_16426.jpg
Image successfully downloaded project_pic/hailey hailey disease_16427.jpg
Image successfully downloaded project_pic/pediculosis lids_16428.jpg
Image successfully downloaded project_pic/psoriasis_16429.jpg
Image successfully downloaded project_pic/behcets disease_16430.jpg
Image successfully downloaded project_pic/tungiasis_16431.jpg
Image successfully downloaded project_pic/pediculosis lids_16432.jpg
Image successfully downloaded proj

Image successfully downloaded project_pic/lichen planus_16538.jpg
Image successfully downloaded project_pic/behcets disease_16539.jpg
Image successfully downloaded project_pic/mycosis fungoides_16540.jpg
Image successfully downloaded project_pic/kaposi sarcoma_16541.jpg
Image successfully downloaded project_pic/acrodermatitis enteropathica_16542.jpg
Image successfully downloaded project_pic/mucinosis_16543.jpg
Image successfully downloaded project_pic/granuloma annulare_16544.jpg
Image successfully downloaded project_pic/aplasia cutis_16545.jpg
Image successfully downloaded project_pic/lupus erythematosus_16546.jpg
Image successfully downloaded project_pic/nematode infection_16547.jpg
Image successfully downloaded project_pic/lupus erythematosus_16548.jpg
Image successfully downloaded project_pic/papilomatosis confluentes and reticulate_16549.jpg
Image successfully downloaded project_pic/ichthyosis vulgaris_16550.jpg
Image successfully downloaded project_pic/urticaria_16551.jpg
Image s

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'Uma ligação estabelecida foi anulada pelo software no computador anfitrião', None, 10053, None))

In [34]:
csv_file = './fitzpatrick17k.csv'  # CSV file path
url_column_name = 'url'
label_column_name = 'label'
output_folder = 'project_pic'  # folder where to save downloaded images
skip_index = 16567
download_images_from_csv(csv_file, url_column_name, output_folder, label_column_name, skip_index)

Image successfully downloaded project_pic/lichen planus_16568.jpg
Image successfully downloaded project_pic/porokeratosis actinic_16569.jpg
Image successfully downloaded project_pic/drug induced pigmentary changes_16570.jpg
Image successfully downloaded project_pic/lichen amyloidosis_16571.jpg
Image successfully downloaded project_pic/scleroderma_16572.jpg
Image successfully downloaded project_pic/pityriasis rosea_16573.jpg
Image successfully downloaded project_pic/myiasis_16574.jpg
Image successfully downloaded project_pic/vitiligo_16575.jpg
Image successfully downloaded project_pic/dariers disease_16576.jpg


# 4. Preprocess images

In [ ]:
def preprocess_image(image, target_size):
    """
    Preprocesses the input image by resizing and normalizing its pixel values.

    Inputs:
        image (PIL.Image): The input image to be preprocessed.
        target_size (tuple): A tuple specifying the target size (height, width) to resize the image.

    Outputs:
        image_array (numpy.ndarray): A numpy array representing the preprocessed image.
    """
    image = image.resize(target_size) # Resize the image
    image_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
    return image_array

# 5. Split the data

In [35]:
def group_images_by_name(source_folder):
    """
    XXX
    
    Inputs:
        source_folder(XXX):  

    Outputs:
        None.
    """
    # Ensure source folder exists
    if not os.path.exists(source_folder):
        print("Source folder doesn't exist.")
        return
    
    # Get a list of all files in the source folder
    files = os.listdir(source_folder)
    
    # Create a dictionary to store file names without numbers
    name_groups = {}
    
    # Iterate over each file
    for file in files:
        # Split file name and extension
        name, ext = os.path.splitext(file)
        
        # Remove numeric characters from file name
        name_without_numbers = ''.join(filter(lambda x: not x.isdigit(), name))
        
        # If name without numbers already exists, append the file to its list
        if name_without_numbers in name_groups:
            name_groups[name_without_numbers].append(file)
        # Otherwise, create a new list with the file
        else:
            name_groups[name_without_numbers] = [file]
    
    # Create folders and move files
    for name, files in name_groups.items():
        # Create folder
        folder_path = os.path.join(source_folder, name)
        os.makedirs(folder_path, exist_ok=True)
        
        # Move files to folder
        for file in files:
            old_file_path = os.path.join(source_folder, file)
            new_file_path = os.path.join(folder_path, file)
            shutil.move(old_file_path, new_file_path)
            print(f"Moved '{file}' to '{name}' folder.")
    
    print('Grouping completed.')

In [36]:
# test function group_images_by_name
source_folder = './project_pic'
group_images_by_name(source_folder)

Moved 'acanthosis nigricans_10065.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10130.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10143.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10274.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10394.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10588.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_1064.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10719.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10746.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10776.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10799.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_10838.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_11098.jpg' to 'acanthosis nigricans_' folder.
Moved 'acanthosis nigricans_11335.jpg' 

Moved 'acne vulgaris_4540.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4561.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4608.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4636.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_472.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4825.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4883.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_4962.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5034.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5037.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5078.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_510.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5111.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5140.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5153.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5174.jpg' to 'acne vulgaris_' folder.
Moved 'acne vulgaris_5200.jpg' to 'acne vulgaris_' folder.

Moved 'acne_16103.jpg' to 'acne_' folder.
Moved 'acne_16277.jpg' to 'acne_' folder.
Moved 'acne_16278.jpg' to 'acne_' folder.
Moved 'acne_1632.jpg' to 'acne_' folder.
Moved 'acne_1643.jpg' to 'acne_' folder.
Moved 'acne_16448.jpg' to 'acne_' folder.
Moved 'acne_1672.jpg' to 'acne_' folder.
Moved 'acne_1892.jpg' to 'acne_' folder.
Moved 'acne_2005.jpg' to 'acne_' folder.
Moved 'acne_2071.jpg' to 'acne_' folder.
Moved 'acne_218.jpg' to 'acne_' folder.
Moved 'acne_2209.jpg' to 'acne_' folder.
Moved 'acne_2229.jpg' to 'acne_' folder.
Moved 'acne_2321.jpg' to 'acne_' folder.
Moved 'acne_247.jpg' to 'acne_' folder.
Moved 'acne_2491.jpg' to 'acne_' folder.
Moved 'acne_2522.jpg' to 'acne_' folder.
Moved 'acne_2729.jpg' to 'acne_' folder.
Moved 'acne_2751.jpg' to 'acne_' folder.
Moved 'acne_2758.jpg' to 'acne_' folder.
Moved 'acne_2900.jpg' to 'acne_' folder.
Moved 'acne_2909.jpg' to 'acne_' folder.
Moved 'acne_3003.jpg' to 'acne_' folder.
Moved 'acne_31.jpg' to 'acne_' folder.
Moved 'acne_3160

Moved 'acrodermatitis enteropathica_12944.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_12960.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_1307.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13193.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13195.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_1333.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13428.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13537.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13544.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13725.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrodermatitis enteropathica_13827.jpg' to 'acrodermatitis enteropathica_' folder.
Moved 'acrod

Moved 'actinic keratosis_2360.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2361.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_240.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2410.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2467.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_261.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2644.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2756.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2766.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2904.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_2937.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_3037.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_3052.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_3195.jpg' to 'actinic keratosis_' folder.
Moved 'actinic keratosis_3216.jpg' to 'actinic keratosis_' folde

...

Moved 'telangiectases_5463.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_5579.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_5580.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_570.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_5897.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_5989.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6073.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6082.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6091.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6179.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_626.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6373.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6694.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6756.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6759.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_6929.jpg' to 'telangiectases_' folder.
Moved 'telangiectases_7086

Moved 'tungiasis_13117.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13137.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13153.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13219.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13225.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13299.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13317.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13365.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13371.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13405.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13409.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13542.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13545.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13558.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13600.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13602.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13636.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13778.jpg' to 'tungiasis_' folder.
Moved 'tungiasis_13858.jpg' to 'tungiasis_' folder.
Moved 'tungi

Moved 'urticaria pigmentosa_9667.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria pigmentosa_9764.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria pigmentosa_9767.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria pigmentosa_9917.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria pigmentosa_9961.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria pigmentosa_9977.jpg' to 'urticaria pigmentosa_' folder.
Moved 'urticaria_10219.jpg' to 'urticaria_' folder.
Moved 'urticaria_10241.jpg' to 'urticaria_' folder.
Moved 'urticaria_1028.jpg' to 'urticaria_' folder.
Moved 'urticaria_10495.jpg' to 'urticaria_' folder.
Moved 'urticaria_1068.jpg' to 'urticaria_' folder.
Moved 'urticaria_10698.jpg' to 'urticaria_' folder.
Moved 'urticaria_1098.jpg' to 'urticaria_' folder.
Moved 'urticaria_11175.jpg' to 'urticaria_' folder.
Moved 'urticaria_11215.jpg' to 'urticaria_' folder.
Moved 'urticaria_11286.jpg' to 'urticaria_' folder.
Moved 'urticaria_11425.jpg' to 'urticaria_' f

Moved 'vitiligo_2760.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_2787.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_2840.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_2842.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_2872.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_3073.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_3256.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_3736.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_3837.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_385.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4042.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4262.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4272.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4412.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4722.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4763.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_479.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_4879.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_5070.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_516.jpg' to 'vitiligo_' folder.
Moved 'vitiligo_53.jpg'

Moved 'xeroderma pigmentosum_17.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_1721.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_1885.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_2121.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_2388.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_2928.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_2942.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_3157.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_3262.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_3544.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_4081.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_4161.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosum_4419.jpg' to 'xeroderma pigmentosum_' folder.
Moved 'xeroderma pigmentosu

In [ ]:
def correct_image_sequence(folder_path):
    """
    XXX
    
    Inputs:
        folder_path(str): 

    Outputs:
        None.
    """
    # Ensure folder exists
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' doesn't exist.")
        return
    
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Sort files alphabetically to maintain order
    files.sort()
    
    # Counter for numbering images
    counter = 1
    
    # Iterate over each file
    for file in files:
        # Split file name and extension
        name, ext = os.path.splitext(file)
        
        # Construct new file name with corrected sequence number
        new_name = f"{name}_{counter}{ext}"
        
        # Rename the file
        old_file_path = os.path.join(folder_path, file)
        new_file_path = os.path.join(folder_path, new_name)
        os.rename(old_file_path, new_file_path)
        print(f"Renamed '{file}' to '{new_name}'.")
        
        # Increment counter
        counter += 1
    
    print('Sequence correction completed.')

In [ ]:
# test fucntion correct_image_sequence
folder_path = './project_pic'
correct_image_sequence(folder_path)

In [ ]:
# Base directory where the images are stored
base_dir = '/path/to/base_directory'
original_dataset_dir = '/path/to/original_dataset_directory'

# Directories for the training, validation, and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Function to create directories for each label
def create_label_directories(root_dir):
    """
    XXX
    
    Inputs:
        folder_path(str): 

    Outputs:
        None.
    """
    for label in labels:
        label_dir = os.path.join(root_dir, label)
        os.mkdir(label_dir)

# Labels
labels = ('drug_induced_pigmentary_changes', 'photodermatoses', 'dermatofibroma', 'psoriasis', 'kaposi_sarcoma',
          'neutrophilic_dermatoses', 'granuloma_annulare', 'nematode_infection', 'allergic_contact_dermatitis',
          'necrobiosis_lipoidica', 'hidradenitis', 'melanoma', 'acne_vulgaris', 'sarcoidosis', 'xeroderma_pigmentosum',
          'actinic_keratosis', 'scleroderma', 'syringoma', 'folliculitis', 'pityriasis_lichenoides_chronica', 'porphyria',
          'dyshidrotic_eczema', 'seborrheic_dermatitis', 'prurigo_nodularis', 'acne', 'neurofibromatosis', 'eczema', 'pediculosis_lids',
          'basal_cell_carcinoma', 'pityriasis_rubra_pilaris', 'pityriasis_rosea', 'livedo_reticularis',
          'stevens_johnson_syndrome', 'erythema_multiforme', 'acrodermatitis_enteropathica', 'epidermolysis_bullosa',
          'dermatomyositis', 'urticaria', 'basal_cell_carcinoma_morpheiform', 'vitiligo', 'erythema_nodosum', 'lupus_erythematosus',
          'lichen_planus', 'sun_damaged_skin', 'drug_eruption', 'scabies', 'cheilitis', 'urticaria_pigmentosa', 'behcets_disease',
          'nevocytic_nevus', 'mycosis_fungoides', 'superficial_spreading_melanoma_ssm', 'porokeratosis_of_mibelli',
          'juvenile_xanthogranuloma', 'milia', 'granuloma_pyogenic', 'papilomatosis_confluentes_and_reticulate',
          'neurotic_excoriations', 'epidermal_nevus', 'naevus_comedonicus', 'erythema_annulare_centrifigum', 'pilar_cyst',
          'pustular_psoriasis', 'ichthyosis_vulgaris', 'lyme_disease', 'striae', 'rhinophyma', 'calcinosis_cutis', 'stasis_edema',
          'neurodermatitis', 'congenital_nevus', 'squamous_cell_carcinoma', 'mucinosis', 'keratosis_pilaris', 'keloid', 'tuberous_sclerosis',
          'acquired_autoimmune_bullous_diseaseherpes_gestationis', 'fixed_eruptions', 'lentigo_maligna', 'lichen_simplex',
          'dariers_disease', 'lymphangioma', 'pilomatricoma', 'lupus_subacute', 'perioral_dermatitis',
          'disseminated_actinic_porokeratosis', 'erythema_elevatum_diutinum', 'halo_nevus', 'aplasia_cutis', 'incontinentia_pigmenti',
          'tick_bite', 'fordyce_spots', 'telangiectases', 'solid_cystic_basal_cell_carcinoma', 'paronychia', 'becker_nevus',
          'pyogenic_granuloma', 'langerhans_cell_histiocytosis', 'port_wine_stain', 'malignant_melanoma', 'factitial_dermatitis',
          'xanthomas', 'nevus_sebaceous_of_jadassohn', 'hailey_hailey_disease', 'scleromyxedema', 'porokeratosis_actinic',
          'rosacea', 'acanthosis_nigricans', 'myiasis', 'seborrheic_keratosis', 'mucous_cyst', 'lichen_amyloidosis',
          'ehlers_danlos_syndrome', 'tungiasis')

# Create directories for each label in train, validation, and test sets
create_label_directories(train_dir)
create_label_directories(validation_dir)
create_label_directories(test_dir)

# Function to copy images to respective directories
def copy_images(source_dir, destination_dir, start_idx, end_idx):
    """
    XXX
    
    Inputs:
        folder_path(str): 

    Outputs:
        None.
    """
    for idx in range(start_idx, end_idx):
        fname = f'{idx}.jpg'
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(destination_dir, fname)
        shutil.copyfile(src, dst)

# Copy images for each label to train, validation, and test sets
for label in labels:
    label_dir = os.path.join(original_dataset_dir, label)
    
    # Training set (first 1000 images)
    train_label_dir = os.path.join(train_dir, label)
    copy_images(label_dir, train_label_dir, 0, 1000)
    
    # Validation set (next 500 images)
    validation_label_dir = os.path.join(validation_dir, label)
    copy_images(label_dir, validation_label_dir, 1000, 1500)
    
    # Test set (next 500 images)
    test_label_dir = os.path.join(test_dir, label)
    copy_images(label_dir, test_label_dir, 1500, 2000)

In [ ]:
# drug induced pigmentary changes
# print()

# photodermatoses

# dermatofibroma

# psoriasis

# kaposi sarcoma

# neutrophilic dermatoses

# granuloma annulare

# nematode infection

# allergic contact dermatitis

# necrobiosis lipoidica

# hidradenitis

# melanoma

# acne vulgaris

# sarcoidosis

# xeroderma pigmentosum

# actinic keratosis

# scleroderma

# syringoma

# folliculitis

# pityriasis lichenoides chronica

# porphyria

# dyshidrotic eczema

# seborrheic dermatitis

# prurigo nodularis

# acne

# neurofibromatosis

# eczema

# pediculosis lids

# basal cell carcinoma

# pityriasis rubra pilaris

# pityriasis rosea

# livedo reticularis

# stevens johnson syndrome

# erythema multiforme

# acrodermatitis enteropathica'

# epidermolysis bullosa

# dermatomyositis

# urticaria

# basal cell carcinoma morpheiform

# vitiligo

# erythema nodosum

# lupus erythematosus

# lichen planus

# sun damaged skin

# drug eruption

# scabies

# cheilitis

# urticaria pigmentosa

# behcets disease

# nevocytic nevus

# mycosis fungoides

# superficial spreading melanoma ssm

# porokeratosis of mibelli

# juvenile xanthogranuloma

# milia

# granuloma pyogenic

# papilomatosis confluentes and reticulate

# neurotic excoriations

# epidermal nevus

# naevus comedonicus

# erythema annulare centrifigum

# pilar cyst

# pustular psoriasis

# ichthyosis vulgaris

# lyme disease

# striae

# rhinophyma

# calcinosis cutis

# stasis edema

# neurodermatitis

# congenital nevus

# squamous cell carcinoma

# mucinosis

# keratosis pilaris

# keloid

# tuberous sclerosis

# acquired autoimmune bullous diseaseherpes gestationis

# fixed eruptions

# lentigo maligna

# lichen simplex

# dariers disease

# lymphangioma

# pilomatricoma

# lupus subacute

# perioral dermatitis

# disseminated actinic porokeratosis

# erythema elevatum diutinum

# halo nevus

# aplasia cutis

# incontinentia pigmenti

# tick bite

# fordyce spots

# telangiectases

# solid cystic basal cell carcinoma

# paronychia

# becker nevus

# pyogenic granuloma

# langerhans cell histiocytosis

# port wine stain

# malignant melanoma

# factitial dermatitis

# xanthomas

# nevus sebaceous of jadassohn

# hailey hailey disease

# scleromyxedema

# porokeratosis actinic

# rosacea

# acanthosis nigricans

# myiasis

# seborrheic keratosis

# mucous cyst

# lichen amyloidosis

# ehlers danlos syndrome

# tungiasis

Check the data:

# 6. Build the model

In [ ]:
# Define the model
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

# Flatten and fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(114, activation='softmax'))  # Output layer, adjust num_classes accordingly

# Print model summary
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss'),  # Stop training if validation loss doesn't improve for 5 epochs
    ModelCheckpoint('best_model.h5', save_best_only=True)  # Save the best model based on validation loss
]

history = model.fit(train_images, train_labels, epochs=num_epochs, 
                    validation_data=(val_images, val_labels),
                    callbacks=callbacks)